In [35]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.window import Window

spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.jars.packages", 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0') \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()


Be sure to start the stream on Kafka!

In [36]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, TimestampType, DateType

schema = StructType(
      [
        StructField("name", StringType(), False),
        StructField("price", DoubleType(), False),
        StructField("timestamp", TimestampType(), False),
      ]
    )


In [37]:
kafka_server = "kafka1:9092"   
from pyspark.sql.functions import from_json

lines = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "stock")                       # Subscribe to the "en" Kafka topic 
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)             # Rate limit on max offsets per trigger interval
  .load()
  .select(from_json(col("value").cast("string"), schema).alias("parsed_value"))
# Load the DataFrame
)
df = lines.select("parsed_value.*")


## The assignment starts here

You can create a

## Implement a control that checks if a stock does not lose too much value in a period of time

In [38]:
from pyspark.sql.functions import window, col
from pyspark.sql import functions as F

# Set the price loss threshold and the name of the stock you are interested in
loss_threshold = 0.05 #5%
# Replace with the name of the stock ticker you want to track
stock_name = "FITB"  

    
# Apply a time window and calculate the price change during that period
windowedDF = df \
    .withWatermark("timestamp", "1 minutes") \
    .filter(col("name") == stock_name) \
    .groupBy(window("timestamp", "10 minutes")) \
    .agg(
        F.expr("min_by(price, timestamp)").alias('first_price'),
        F.expr("max_by(price, timestamp)").alias('last_price'),
        F.max('timestamp').alias('lastTimeStamp'),
        F.min('timestamp').alias('firstTimeStamp')
        ) 


# Check if the price change exceeds the threshold
result_df = windowedDF.withColumn("percent_value_lost", ((col("first_price") - col("last_price"))/col("first_price")))
result_final = result_df.filter((col("last_price") < col("first_price")) & 
                                (col("percent_value_lost") > loss_threshold))

# Save and show
query = (result_final.writeStream
         .outputMode("complete")
         .option("truncate", False)
         .format("console")
         .start())


23/11/06 19:53:38 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-748a0de4-165c-413e-b160-64cb198f205c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+-----------+----------+-------------+--------------+------------------+
|window|first_price|last_price|lastTimeStamp|firstTimeStamp|percent_value_lost|
+------+-----------+----------+-------------+--------------+------------------+
+------+-----------+----------+-------------+--------------+------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----------+----------+-------------+--------------+------------------+
|window|first_price|last_price|lastTimeStamp|firstTimeStamp|percent_value_lost|
+------+-----------+----------+-------------+--------------+------------------+
+------+-----------+----------+-------------+--------------+------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+-----------+----------+-------------+--------------+------------------+
|window|first_price|last_price|lastTimeStamp|firstTimeStamp|percent_value_lost|
+------+-----------+----------+-------------+--------------+------------------+
+------+-----------+----------+-------------+--------------+------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+------+-----------+----------+-------------+--------------+------------------+
|window|first_price|last_price|lastTimeStamp|firstTimeStamp|percent_value_lost|
+------+-----------+----------+-------------+--------------+------------------+
+------+-----------+----------+-------------+--------------+------------------+



...

-------------------------------------------
Batch: 129
-------------------------------------------
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+
|window                                    |first_price|last_price|lastTimeStamp      |firstTimeStamp     |percent_value_lost |
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+
|[2023-11-05 12:10:00, 2023-11-05 12:20:00]|20.67      |19.525    |2023-11-05 12:17:23|2023-11-05 12:15:14|0.05539429124334799|
|[2023-11-05 12:20:00, 2023-11-05 12:30:00]|18.99      |17.48     |2023-11-05 12:29:47|2023-11-05 12:24:16|0.07951553449183771|
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+



-------------------------------------------
Batch: 130
-------------------------------------------
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+
|window                                    |first_price|last_price|lastTimeStamp      |firstTimeStamp     |percent_value_lost |
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+
|[2023-11-05 12:10:00, 2023-11-05 12:20:00]|20.67      |19.525    |2023-11-05 12:17:23|2023-11-05 12:15:14|0.05539429124334799|
|[2023-11-05 12:20:00, 2023-11-05 12:30:00]|18.99      |17.48     |2023-11-05 12:29:47|2023-11-05 12:24:16|0.07951553449183771|
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+



-------------------------------------------
Batch: 131
-------------------------------------------
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+
|window                                    |first_price|last_price|lastTimeStamp      |firstTimeStamp     |percent_value_lost |
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+
|[2023-11-05 12:10:00, 2023-11-05 12:20:00]|20.67      |19.525    |2023-11-05 12:17:23|2023-11-05 12:15:14|0.05539429124334799|
|[2023-11-05 12:20:00, 2023-11-05 12:30:00]|18.99      |17.48     |2023-11-05 12:29:47|2023-11-05 12:24:16|0.07951553449183771|
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+



-------------------------------------------
Batch: 132
-------------------------------------------
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+
|window                                    |first_price|last_price|lastTimeStamp      |firstTimeStamp     |percent_value_lost |
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+
|[2023-11-05 12:10:00, 2023-11-05 12:20:00]|20.67      |19.525    |2023-11-05 12:17:23|2023-11-05 12:15:14|0.05539429124334799|
|[2023-11-05 12:20:00, 2023-11-05 12:30:00]|18.99      |17.48     |2023-11-05 12:29:47|2023-11-05 12:24:16|0.07951553449183771|
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+



-------------------------------------------
Batch: 133
-------------------------------------------
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+
|window                                    |first_price|last_price|lastTimeStamp      |firstTimeStamp     |percent_value_lost |
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+
|[2023-11-05 12:10:00, 2023-11-05 12:20:00]|20.67      |19.525    |2023-11-05 12:17:23|2023-11-05 12:15:14|0.05539429124334799|
|[2023-11-05 12:20:00, 2023-11-05 12:30:00]|18.99      |17.48     |2023-11-05 12:29:47|2023-11-05 12:24:16|0.07951553449183771|
+------------------------------------------+-----------+----------+-------------------+-------------------+-------------------+



In [31]:
# Set the price loss threshold and the name of the stock you are interested in
loss_threshold = 0.05 #5%
# Replace with the name of the stock ticker you want to track
stock_name = "FITB"  

# Define the start and end time for the time range you want to check
start_time = "2023-11-05 12:10:00"
end_time = "2023-11-05 12:30:00"

# Apply a time window and calculate the price change during that period
windowedDF = df \
    .withWatermark("timestamp", "1 minutes") \
    .filter((col("name") == stock_name) & (col("timestamp").between(start_time, end_time))) \
    .agg(
        F.expr("min_by(price, timestamp)").alias('first_price'),
        F.expr("max_by(price, timestamp)").alias('last_price'),
        F.max('timestamp').alias('lastTimeStamp'),
        F.min('timestamp').alias('firstTimeStamp')
    ) 


# Check if the price change exceeds the threshold
result_df = windowedDF.withColumn("percent_value_lost", ((col("first_price") - col("last_price"))/col("first_price")))
result_final = result_df.filter((col("last_price") < col("first_price")) & 
                                (col("percent_value_lost") > loss_threshold))

# Save and show
query = (result_final.writeStream
         .outputMode("complete")
         .option("truncate", False)
         .format("meromy")
         .queryName("checking1")
         .start())
